In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# MES AÑO DE CONCILIACIÓN

In [2]:
month_conc = 'April'
year_conc = '2022'

# RUTAS DE ACCESO

### SWAP DEL DIA 

In [3]:
allfiles_disp_trad = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\04. ABRIL\ARCHIVO DISPERSIONES\Dispersión*.xlsx', recursive=False)
allfiles_disp_trad

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-2-20220424.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220401.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220402.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220403.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220404.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220405.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220406.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES\\Dispersión-20220407.xlsx',
 'C:\\Users\\C

### FAST PAYMENTS

In [4]:
allfiles_fp = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\04. ABRIL\ARCHIVO DISPERSIONES FP\Dispersión-FP*.xlsx',recursive=False)
allfiles_fp

['C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-01.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-03.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-04.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-05.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-06.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\ARCHIVO DISPERSIONES FP\\Dispersión-FP 2022-04-07.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2022\\04. ABRIL\\A

# SWAP DEL DIA

In [ ]:
swap_dia_total = pd.concat([pd.read_excel(f,sheet_name = 'Swap del día')
                            .assign(file_name=os.path.basename(f)) for f in allfiles_disp_trad])
swap_dia_total['Date'] = swap_dia_total["file_name"].str[-13:-5]
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'], format='%Y%m%d').dt.date
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'])
swap_dia_total.sort_values(by=['Date'],ascending=True,inplace=True)
swap_dia_total['report_date_local']= pd.to_datetime(swap_dia_total['report_date_local'])
swap_dia_total['month_conc'] = month_conc
swap_dia_total['year_conc'] = year_conc
swap_dia_total['month_report'] = swap_dia_total['report_date_local'].dt.strftime('%B')
swap_dia_total['year_report'] = swap_dia_total['report_date_local'].dt.year.astype(str)
swap_dia_total.head()

In [ ]:
swap_dia_total.shape

In [ ]:
swap_dia_total.dtypes

In [ ]:
pivot_total = pd.pivot_table(swap_dia_total,index=['Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_total.reset_index(drop=False, inplace=True)
pivot_total.head()

In [ ]:
pivot_parcial = pd.pivot_table(swap_dia_total,index=['tipo','Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_parcial.reset_index(drop=False, inplace=True)
pivot_parcial.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES

In [ ]:
swap_dia_total_repro = swap_dia_total[(swap_dia_total['tipo'] == 'Reprogramación')]
swap_dia_total_repro.head()

In [ ]:
swap_dia_total_repro.shape

In [ ]:
swap_dia_total_repro.loc[(swap_dia_total_repro['tipo'] == 'Reprogramación')
                   & (swap_dia_total_repro['month_report'] == month_conc) 
                   & (swap_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
swap_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
swap_dia_total_repro.head()

In [ ]:
pivot_tipo_repro = pd.pivot_table(swap_dia_total_repro,index=['Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro.reset_index(drop=False, inplace=True)
pivot_tipo_repro.head()

# FAST PAYMENTS

In [ ]:
fp_dia_total = pd.concat([pd.read_excel(f,sheet_name='FAST PAYMENTS')
                          .assign(file_name=os.path.basename(f)) for f in allfiles_fp])
fp_dia_total['Date'] = fp_dia_total["file_name"].str[-15:-5]
fp_dia_total['Date']= pd.to_datetime(fp_dia_total['Date'])
fp_dia_total['report_date_local']= pd.to_datetime(fp_dia_total['report_date_local'])
fp_dia_total['month_conc'] = month_conc
fp_dia_total['year_conc'] = year_conc
fp_dia_total['month_report'] = fp_dia_total['report_date_local'].dt.strftime('%B')
fp_dia_total['year_report'] = fp_dia_total['report_date_local'].dt.year.astype(str)
fp_dia_total.head()

In [ ]:
fp_dia_total.shape

In [ ]:
fp_dia_total.dtypes

In [ ]:
pivot_fp_total = pd.pivot_table(fp_dia_total,index=['Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_fp_total.reset_index(drop=False, inplace=True)
pivot_fp_total.head()

In [ ]:
pivot_fp_parcial = pd.pivot_table(fp_dia_total,index=['tipo','Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_fp_parcial.reset_index(drop=False, inplace=True)
pivot_fp_parcial.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES - FAST PAYMENTS

In [ ]:
fp_dia_total_repro = fp_dia_total[(fp_dia_total['tipo'] == 'Reprogramación')]
fp_dia_total_repro.head()

In [ ]:
fp_dia_total_repro.shape

In [ ]:
fp_dia_total_repro.loc[(fp_dia_total_repro['tipo'] == 'Reprogramación')
                   & (fp_dia_total_repro['month_report'] == month_conc) 
                   & (fp_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
fp_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
fp_dia_total_repro.head()

In [ ]:
pivot_tipo_repro_fp = pd.pivot_table(fp_dia_total_repro,index=['Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro_fp.reset_index(drop=False, inplace=True)
pivot_tipo_repro_fp.head()

# EXCEL FINAL

In [ ]:
writer = pd.ExcelWriter('FASE IV. REPROGRAMACIONES ABRIL 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_total.to_excel(writer, sheet_name='SWAP DIA TOTAL')
pivot_parcial.to_excel(writer, sheet_name='SWAP DIA PARCIAL')
pivot_fp_total.to_excel(writer, sheet_name='FP TOTAL')
pivot_fp_parcial.to_excel(writer, sheet_name='FP PARCIAL')
pivot_tipo_repro.to_excel(writer, sheet_name='CATEGORIZACION REPRO')
pivot_tipo_repro_fp.to_excel(writer, sheet_name='CATEGORIZACION FP-REPRO')
writer.close()